# ==================

Constructing Graph and Adjacencies

In [78]:
class Vertex:
  def __init__(self, graphLetter, graphNum, indexInGraph):
    self.graphLetter = graphLetter
    self.graphNum = graphNum
    self.indexInGraph = indexInGraph

    self.label = None
    self.name = f"({graphLetter}, {graphNum}, {indexInGraph})"
    self.nameTuple = (graphLetter, graphNum, indexInGraph)

  def __repr__(self):
    return f"Vertex({self.name}, label={self.label})"

  def __eq__(self, other):
    return ( self.graphLetter == other.graphLetter and self.graphNum == other.graphNum and self.indexInGraph == other.indexInGraph )

  def __lt__(self, other):
    return self.nameTuple < other.nameTuple

  def __hash__(self):
    return hash((self.graphLetter, self.graphNum, self.indexInGraph))

In [79]:
Adjacencies = {}
Verticies = {}

In [80]:
def AddAdjacency(u, v):
  global Adjacencies
  if u == v or (v in Adjacencies[u]):
    return
  Adjacencies[u].append(v)
  Adjacencies[v].append(u)


def MergeGraphs(vToMerge, vToRemove):
  global Adjacencies

  prevNeighbors = Adjacencies[vToRemove]
  for neighbor in prevNeighbors:
    if neighbor != vToMerge:
      AddAdjacency(vToMerge, neighbor)
    Adjacencies[neighbor].remove(vToRemove)

  del Adjacencies[vToRemove]
  del Verticies[vToRemove.nameTuple]

In [81]:
def C(n,r,m,inner):
  global Adjacencies, Verticies
  Adjacencies = {}
  Verticies = {}

  # Initialize Verticies
  for j in range(m):
    for k in range(n):
      Verticies[("A", j, k)] = Vertex("A", j, k)
      Verticies[("B", j, k)] = Vertex("B", j, k)
      Adjacencies[Verticies[("A", j, k)]] = []
      Adjacencies[Verticies[("B", j, k)]] = []

  # Initialize Connectivities of Graph A and Graph B
  for j in range(m):
    for k in range(n):

      # Connect Pentagon
      u = Verticies[("A", j, k)]
      v = Verticies[("A", j, (k + 1) % n)]
      AddAdjacency(u, v)

      # Connect Star
      b = Verticies[("B", j, k)]
      bLeft = Verticies[("B", j, (k + r) % n)]
      bRight = Verticies[("B", j, (k - r) % n)]
      AddAdjacency(b, bLeft)
      AddAdjacency(b, bRight)

      # Connect Pentagon to Star (Outer / Inner)
      AddAdjacency(Verticies[("A", j, k)], Verticies[("B", j, k)])


  index = n // 2

  for j in range(m - 1):
    if inner:
      vToMerge = Verticies[("A", j, index)]
      vToRemove = Verticies[("A", j + 1, 0)]
    else:
      vToMerge = Verticies[("B", j, index)]
      vToRemove = Verticies[("B", j + 1, 0)]

    MergeGraphs(vToMerge, vToRemove)

  for vertex, adjacencies in list(Adjacencies.items()):
    Adjacencies[vertex] = sorted(adjacencies, key=lambda v: (v.graphNum, v.graphLetter, v.indexInGraph), reverse=(not inner))

In [82]:
for vert in sorted(Verticies):
  print(f'{Verticies[vert]}  :  {Adjacencies[Verticies[vert]]}\n' )

# ============

Code for assigning labels:

In [83]:
from collections import deque

In [84]:
next = deque()
fel = set()             #Final Edge List (weights)
unusedEdge = deque()
felMax = 0   #Gloabl Edge Weight Maximum

In [85]:
def AddWeight(weight):
  global felMax, fel, unusedEdge
  fel.add(weight)
  if weight > felMax:
    for i in range(felMax+1, weight):
      unusedEdge.append(i)
    felMax = weight

def UpdateEdges(sourceVertex, vertexToUpdate):
  global unusedEdge
  for adj in Adjacencies[vertexToUpdate]:
    if not adj == sourceVertex and adj.label is not None:
      w = vertexToUpdate.label + adj.label
      AddWeight(w)
      if w in unusedEdge:
        unusedEdge.remove(w)

def UpdateVertex(sourceVertex, vertexToUpdate, label, weight, src=""):
  vertexToUpdate.label = label
  AddWeight(weight)
  UpdateEdges(sourceVertex, vertexToUpdate)

def isAcceptibleLabel(examinee, consideredLabel):
  for adj in Adjacencies[examinee]:
    if adj.label is not None:
      if (consideredLabel + adj.label) in fel:
        return False
    for adjadj in Adjacencies[adj]:
      if adjadj.label == consideredLabel:
        return False
  return True

In [86]:

def AssignLabels(startVertex):
  global next, fel, unusedEdge, felMax
  next = deque()
  fel = set()             #Final Edge List (weights)
  unusedEdge = deque()
  felMax = 0  #Gloabl Edge Weight Maximum


  startVertex.label = 1
  firstAdj = Adjacencies[startVertex][0]
  firstAdj.label = 1

  next.append(startVertex)
  next.append(firstAdj)
  fel.add(2)
  felMax = 2

  # Main Loop
  while next:
    Source = next.popleft()

    for Examinee in Adjacencies[Source]:
      if Examinee.label is None:
        next.append(Examinee)

        for weight in list(unusedEdge):
          consideredWeight = weight
          consideredLabel = consideredWeight - Source.label


          if isAcceptibleLabel(Examinee, consideredLabel):
            UpdateVertex(Source, Examinee, consideredLabel, consideredWeight, "[from unusedEdge]")
            unusedEdge.remove(weight)
            break

        if Examinee.label is not None:
          continue

        consideredWeight = felMax + 1

        while not isAcceptibleLabel(Examinee, consideredWeight - Source.label):
          consideredWeight += 1

        consideredLabel = consideredWeight - Source.label
        UpdateVertex(Source, Examinee, consideredLabel, consideredWeight, "[new weight]")

  return fel, list(unusedEdge)


# ==============

Testing Code:



NOTE : You need to reconstruct the graph every time. This is to clear any old labels. I keep the command seperate in case we need to log how long the actual labeling part takes as opposed to the labeling and the constucting of the graphs.

In [ ]:
# Contruct graphs
C(n=5, r=2, m=10000, inner=False)


In [106]:
fel,unusedEdges = AssignLabels(Verticies[("A",0,0)])
print(f"\n\n\nNumber of Verticies : {len(Verticies.keys())}")
#print(f"Final Edge List : {fel}")
print(f"Final Edge List Max : {felMax}")
#print(f"Unused Edges : {unusedEdges}")
print(f"Unused Edges Len : {len(unusedEdges)}")




Number of Verticies : 90001
Final Edge List Max : 155382
Unused Edges Len : 5381


In [107]:
get_max_label = lambda d: max(v.label for v in d.values())
print(f'Max label : {get_max_label(Verticies)}')

Max label : 77692
